***Busqueda de N1 min y max***
$$B_{max} = \frac{1}{2}\frac{\int V_L(t)dt}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$
$$\frac{1}{2}\frac{V_dDT_s}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$

Con

$$R = \frac{l_e}{\mu_0 \mu_e A}$$

Desmos: https://www.desmos.com/calculator/x80nw1l7ab

Cosas de TME:
$$N_{min} = \frac{1}{2}\frac{V_dDT_s}{B_{sat}A_{min}}$$

In [4]:
from scipy.optimize import fsolve
import numpy as np

def get_turns(Vd, Vo2, Vo3, D_mean, N1, Al):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2
    print("")
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    print("\tNormalizando:")
    N2 = np.round(N2)
    L2 = Al*N2**2
    N3 = np.round(N3)
    L3 = Al*N3**2
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    Vo2 = Vd*(N2/N1)*(D_mean/(1-D_mean))
    Vo3 = Vd*(N3/N1)*(D_mean/(1-D_mean))
    print("\tVo2 = ", Vo2, "V")
    print("\tVo3 = ", Vo3, "V")
    print("")
    return

Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 280
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.45    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
#Corriente DC maxima para el MOS
IDC = 2
#Datos del nucleo
Al = 5200E-9
Al_min = Al*(1-0.2)
Al_max = Al*(1+0.3)
A_min = 229E-6
le = 97E-3
u0 = 4*np.pi*1E-7
ue = 1690
Ae = 234E-6
R = le/(u0*ue*A_min)
#Si usamos un gap:
lgap = 0.2E-3 #Probamos con gap de medio milimetro
R = lgap/(u0*Ae)

#Ahora encontramos los parametros limite y deseados
N_min = 0.5*Vd*D*Ts/(Bsat*A_min)
print("N_min = ", N_min, " vueltas. Redondeamos a ", np.ceil(N_min))

#Este es el N_min para que el nucleo no sature por su propia inductancia, ahora tenemos que ver un Idc maximo admitible
#Resolvemos la ecuacion cuadratica: 0.5*Vd*D*Ts/(A_min) + (N**2)*IDC/(R*A_min) - Bsat*N = 0
coeffs = [IDC*Al/(A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("Raices:", np.roots(coeffs), ",\t Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo")
#Vemos que nos da raices complejas, asi que la IDC (que es el unico parametro que podemos mover, es muy alta)

#Resto de parametros del transformador
Vo2 = 24+1 #+1Vdc para compensar caida en diodos
Vo3 = 16+1


print("Distintas opciones de vueltas para distintos rangos de IDC:\n")
print("Opcion 1:\n")
IDC = 250E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = ", IDC*1E3, "mA")
N1 = 132
get_turns(Vd, Vo2, Vo3, 0.30, N1, 1/R)

N_min =  16.6732830488289  vueltas. Redondeamos a  17.0
Raices: [2.42211538+8.65462812j 2.42211538-8.65462812j] ,	 Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo
Distintas opciones de vueltas para distintos rangos de IDC:

Opcion 1:

	Raices: [117.63696052  19.42673235] ,	 Con IDC =  250.0 mA

	N1 =  132  vueltas
	L1 =  25.617903665397524 mHy
	N2 =  27.5  vueltas
	L2 =  1.1118881799217675 mHy
	N3 =  18.7  vueltas
	L3 =  0.5141370943958252 mHy

	Normalizando:
	N1 =  132  vueltas
	L1 =  25.617903665397524 mHy
	N2 =  28.0  vueltas
	L2 =  1.1526880437139382 mHy
	N3 =  19.0  vueltas
	L3 =  0.5307657956386884 mHy

	Vo2 =  25.454545454545457 V
	Vo3 =  17.272727272727273 V



In [5]:
##Escribir todas las posibilidades de N1 vueltas a un xml
import pandas as pd

def save_turns(Vd, Vo2, Vo3, D_mean, N1, Al, dataframe):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2

    N2_norm = np.round(N2)
    L2_norm = Al*N2**2
    N3_norm = np.round(N3)
    L3_norm = Al*N3**2

    Vo2_norm = Vd*(N2_norm/N1)*(D_mean/(1-D_mean))
    Vo3_norm = Vd*(N3_norm/N1)*(D_mean/(1-D_mean))
    # add row to the DataFrame
    dataframe.loc[len(dataframe)] = [Vd, Vo2, Vo3, Al*1E9, N1, L1*1E3, N2, L2*1E6, N3, L3*1E6, ' ', N2_norm, L2_norm*1E6, N3_norm, L3_norm*1E6, Vo2_norm, Vo3_norm]
    return dataframe


Al = 1/R
# create an empty DataFrame
df = pd.DataFrame(columns=['Vd', 'Vo2', 'Vo3', 'AL[nHy]', 'N1', 'L1[mHy]', 'N2', 'L2[uHy]', 'N3', 'L3[uHy]', 'Normalizamos...', 'N2_norm', 'L2_norm[uHy]', 'N3_norm', 'L3_norm[uHy]', 'Vo2_norm', 'Vo3_norm'])

# add some rows to the DataFrame
for N1 in range(110, 140, 2):
    df = save_turns(Vd, Vo2, Vo3, 0.30, N1, Al, df)

print(df.to_string(index=False, max_colwidth=20, float_format='%.3f'))

 Vd  Vo2  Vo3  AL[nHy]  N1  L1[mHy]     N2  L2[uHy]     N3  L3[uHy] Normalizamos...  N2_norm  L2_norm[uHy]  N3_norm  L3_norm[uHy]  Vo2_norm  Vo3_norm
280   25   17 1470.265 110   17.790 22.917  772.145 15.583  357.040                   23.000       772.145   16.000       357.040    25.091    17.455
280   25   17 1470.265 112   18.443 23.333  800.478 15.867  370.141                   23.000       800.478   16.000       370.141    24.643    17.143
280   25   17 1470.265 114   19.108 23.750  829.322 16.150  383.478                   24.000       829.322   16.000       383.478    25.263    16.842
280   25   17 1470.265 116   19.784 24.167  858.676 16.433  397.052                   24.000       858.676   16.000       397.052    24.828    16.552
280   25   17 1470.265 118   20.472 24.583  888.541 16.717  410.861                   25.000       888.541   17.000       410.861    25.424    17.288
280   25   17 1470.265 120   21.172 25.000  918.916 17.000  424.907                   25.000       9

In [7]:
#Corrientes en el trafo
Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 280
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.4    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
Al = 5200E-9
N1 = 150
N2 = 32
N3 = 22
D_max = 0.41

print("\nCorrientes maximas para no saturar al trafo")

IN1_DC_max = (A_min*R/N1)*(Bsat-0.5*Vd*D_max*Ts/(N1*A_min))
print("IN1_DC_max = ", IN1_DC_max*1E3, "mA")

IN2_DC_max = IN1_DC_max*N1/N2
IN3_DC_max = IN1_DC_max*N1/N3

print("IN2_DC_max = ", IN2_DC_max*1E3, "mA")
print("R2min = ", Vo2/IN2_DC_max, "ohm")
print("IN3_DC_max = ", IN3_DC_max*1E3, "mA")
print("R3min = ", Vo3/IN3_DC_max, "ohm")

print("\nCorrientes de boundary")
N1 = 132
N2 = 28
N3 = 19
L1 = 25.618E-3
L2 = 1111.888E-6
L3 = 514.7E-6
Iob2 = 0.5*Vo2*(Ts/L2)*((1-0.35)**2)
Iob1 = Iob2*N2/N1
R2MAX = Vo2/Iob2
Iob3 = 0.5*Vo3*(Ts/L3)*((1-0.35)**2)
R3MAX = Vo3/Iob3
print("Iob1 = ", Iob1*1E3, "mA")
print("Iob2 = ", Iob2*1E3, "mA")
print("R2max = ", R2MAX, "ohm")
print("Iob2 = ", Iob3*1E3, "mA")
print("R3max = ", R3MAX, "ohm")



Corrientes maximas para no saturar al trafo
IN1_DC_max =  205.3043346310458 mA
IN2_DC_max =  962.3640685830272 mA
R2min =  25.977694737512056 ohm
IN3_DC_max =  1399.8022815753125 mA
R3min =  12.144572289786886 ohm

Corrientes de boundary
Iob1 =  13.433788912973872 mA
Iob2 =  63.330719161162534 mA
R2max =  394.7531360946744 ohm
Iob2 =  93.03153940806943 mA
R3max =  182.73372781065086 ohm


***Snubber***
$$V_{DS} > V_{C_{SN}} > V_d + V_{o}\frac{N1}{N2}$$

$$C_{snubber} > \frac{L_{disp}*I_{L_{disp}}}{V_{sw_{max}^2 - (V_{d} + V_o \frac{N1}{N2})^2}}$$

$$R_{snubber} < \frac{T_{ON_{min}}}{3C_{snubber}} = \frac{D_{min}Ts}{3C_{snubber}}$$

In [9]:
#Estimacion de Ldisp REVISAR
Ldisp = 0.05*L1
print("Ldisp = ", Ldisp*1E6, "mHy")
ILdisp = IN1_DC_max+Vd*0.5*Ts/Ldisp

Vsw_max = 900

Csn_min = Ldisp*ILdisp/(Vsw_max**2 - (Vd+Vo2*N1/N2)**2)
print("Csn_min = ", Csn_min*1E9, "nF")
Csn = 10E-9
print("Csn = ", Csn*1E9, "nF")
#Ahora para Rsn
Dmin = 0.05
Rsn_max = Dmin*Ts/(3*Csn)
print("Rsn_max = ", Rsn_max, "ohm")

Ldisp =  1280.8999999999999 mHy
Csn_min =  3.267775527819153 nF
Csn =  10.0 nF
Rsn_max =  22.22222222222222 ohm
